In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [3]:
data_path = r'D:\Coding\Python Programming\Iris.csv'
data = pd.read_csv(data_path)

In [4]:
inputs = data[['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm']]

y = data[['Species']]
targets = []

for i in y.values:
    if i == "Iris-setosa":
        targets.append([1,0,0])
    elif i == "Iris-versicolor":
        targets.append([0,1,0])
    else:
        targets.append([0,0,1])

inputs = np.array(inputs)

targets = np.array(targets)


In [5]:
x_train, x_test, y_train, y_test =  train_test_split(inputs, targets, test_size= 0.2)

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)
y_train = sc.fit_transform(y_train)
y_test = sc.fit_transform(y_test)

x_train = torch.from_numpy(x_train.astype(np.float32))
x_test = torch.from_numpy(x_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

torch.Size([120, 4])

In [6]:
class IrisModel(nn.Module):
    def __init__(self):
        super(IrisModel, self).__init__()
        self.linear = nn.Linear(4,3)
    
    def forward(self, x): 
        preds = torch.nn.functional.softmax(self.linear(x), dim = 1)
        return preds

model = IrisModel()
loss_fn = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr= 0.01)


In [7]:
epochs = 10000

for epoch in range(epochs):
    prediction = model(x_train)
    loss = loss_fn(prediction, y_train)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    if (epoch)%1000 == 0:
        print(loss)

preds_arr = torch.argmax(prediction, 1)
targets_arr = torch.argmax(y_train, 1)
with torch.no_grad():
    print("Accuracy: {:.2f}% ".format((torch.sum(preds_arr == targets_arr).item() / len(preds_arr))* 100))
    

tensor(0.7096, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.2543, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.2077, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.1790, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.1583, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.1424, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.1300, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.1201, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.1119, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.1051, grad_fn=<BinaryCrossEntropyBackward>)
Accuracy: 96.67% 


In [8]:
with torch.no_grad():
    test_pred = model(torch.tensor(np.array([[5.5,2.3,4,1.3]]).astype(np.float32)))
test_pred

tensor([[1.5672e-11, 8.0105e-05, 9.9992e-01]])